# 6 Comparative analysis

In [2]:
! pip install .

Processing /home/jovyan/work/latent-dirichlet-allocation
  Running setup.py bdist_wheel for lda-package ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/55/df/03/5c2615a45fff840cc1dac92c4e9d5fa0e8c00f0d88616624ee
Successfully built lda-package
  Found existing installation: lda-package 1.0
    Uninstalling lda-package-1.0:
      Successfully uninstalled lda-package-1.0


In [38]:
import lda_package
import numpy as np
from scipy.special import digamma, polygamma
import matplotlib.pyplot as plt 
import gensim

In [13]:
def perplexity(N, M, gamma, phi):
    
    """
    A method to evaluate models with held-out set.
    
    """
    preplexity = 0.0
    for m in range(M):
        for w in range(N[m]):
            preplexity += np.log(np.sum(gamma[m,:] * phi[m][w,:]))
    return np.exp(-(preplexity / np.sum(N)))
    
    

In [40]:
doc_a = """TheWilliam Randolph Hearst Foundation will give 1.25 million to Lincoln Center, Metropolitan
        Opera Co., New York Philharmonic and Juilliard School."""
doc_b = """ “Our board felt that we had a real opportunity to make a mark on the future of the performing 
           arts with these grants an act every bit as important as our traditional areas of support in health, 
           medical research, education and the social services,” Hearst Foundation President Randolph A. Hearst 
           said Monday in announcing the grants."""
doc_c = """Lincoln Center’s share will be 200000 for its new building, which
           will house young artists and provide new public facilities."""
doc_d = """The Metropolitan Opera Co. and New York Philharmonic will receive 400000 each."""
doc_e = """The Juilliard School, where music and the performing arts are taught, will get 250000. """
doc_f = """The Hearst Foundation, a leading supporter of the Lincoln Center Consolidated Corporate Fund, 
           will make its usual annual $100,000 donation, too."""

doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e, doc_f]

In [97]:
f = open('stop_word.txt', 'r')
stop_word = f.read().splitlines()
f.close()
stop_word = [word.strip() for word in stop_word]
texts, dictionary, corpus = lda_package.data_clean(doc_set, stop_word)
text_ = lda_package.data_process(texts, dictionary)

In [98]:
np.random.seed(64528)

M = 6
k = 4
N = np.array(list(map(len, text_)))
V = len(dictionary)
V_words = range(V)
alpha = np.random.dirichlet(10*np.ones(k),1)[0]
beta = np.random.dirichlet(np.ones(V),k)

phi = np.array([1/k*np.ones([N[m],k]) for m in range(M)])
gamma = np.tile(alpha,(M,1)) + np.tile(N/k,(k,1)).T

In [109]:
#LSI
lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=4)

In [110]:
lsi.print_topics(num_topics=4, num_words=5)

[(0,
  '0.431*"hearst" + 0.322*"grant" + 0.270*"foundat" + 0.217*"randolph" + 0.214*"support"'),
 (1,
  '-0.354*"center" + -0.354*"lincoln" + -0.212*"york" + -0.212*"metropolitan" + -0.212*"opera"'),
 (2,
  '0.261*"york" + 0.261*"opera" + 0.261*"philharmon" + 0.261*"metropolitan" + 0.220*"school"'),
 (3,
  '0.304*"share" + 0.304*"artist" + 0.304*"200000" + 0.304*"build" + 0.304*"facil"')]

In [83]:
#LDA
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word = dictionary)

In [93]:
lda.print_topics(num_topics=4, num_words=4)

[(0, '0.050*"will" + 0.032*"hearst" + 0.031*"co" + 0.031*"opera"'),
 (1, '0.035*"hearst" + 0.028*"grant" + 0.023*"real" + 0.023*"presid"'),
 (2, '0.036*"hearst" + 0.035*"will" + 0.034*"school" + 0.034*"juilliard"'),
 (3, '0.062*"will" + 0.062*"new" + 0.035*"center" + 0.035*"lincoln"')]